In [1]:
import av
import math
from video_a import Video
path = '../videos/SOX5yA1l24A.mp4'

In [2]:
bla = Video(path, stream="video")

In [3]:
bla.seek(10)

In [4]:
print(bla.next())

(array([[ 22,  36,  38, ...,  96,  95,  94],
       [ 22,  36,  38, ..., 100,  98,  98],
       [ 22,  36,  38, ..., 102, 100, 100],
       ...,
       [132, 132, 132, ..., 140, 140, 140],
       [132, 132, 132, ..., 133, 136, 137],
       [132, 132, 132, ..., 128, 130, 131]], dtype=uint8), 9.743066666666667)


# Implementing rudimentary read video function

In [31]:
import torch
from torchvision import transforms as t


# read video
def _read_video(vo,per_frame_transform, start=0, end=None ):
    if not isinstance(vo, Video):
        vo = Video(path)
    
    if end is None:
        end = float("inf")
    
    if end < start:
        raise ValueError(
            "end_pts should be larger than start_pts, got "
            "start_pts={} and end_pts={}".format(start_pts, end_pts)
        )
    
    
    current_pts = start
    frames = []
       
    # this should get us close to the actual starting point we want
    vo.seek(start)
    while current_pts <= end:
        frame, current_pts = vo.next()
        if current_pts >= start and current_pts <= end:
            frames.append(per_frame_transform(frame))
        
    
    return torch.stack(frames, 0)
    

In [39]:
def read_video(vo, start=0, end=None, width=-1, height=-1):
    # get transfroms per frames
    transforms = [t.ToTensor()]
    if width > 0 and height>0:
            transforms.insert(0, t.Resize((height, width), interpolation=2))
            transforms.insert(0, t.ToPILImage())
            
    transform = t.Compose(transforms)
    
    return _read_video(vo, transform, start, end)


In [40]:
test = read_video(bla, 4, 8)
print(test.size())

torch.Size([120, 1, 384, 340])


In [41]:
test = read_video(bla, 8, 10, 376, 224)
print(test.size())

torch.Size([60, 1, 224, 376])


In [42]:
import torch
import torchvision
torchvision.set_video_backend("pyav")
def get_tv(path):
    vframes, _, _ = torchvision.io.read_video(path, 8, 10, pts_unit="sec")
    print("TVAV", len(vframes))

get_tv(path)

TVAV 61
